In [1]:
import heapq
import functools
import collections
import math
import bisect
from typing import *

# 6291. Difference Between Element Sum and Digit Sum of an Array

In [2]:
class Solution:
    def differenceOfSum(self, nums: List[int]) -> int:
        sums = sum(nums)
        digits = 0
        for num in nums:
            while num:
                digits += num % 10
                num //= 10
        return abs(sums-digits)

# 6292. Increment Submatrices by One

In [3]:
class Solution:
    def rangeAddQueries(self, n: int, queries: List[List[int]]) -> List[List[int]]:
        ret = [[0] * n for _ in range(n)]
        for query in queries:
            ret[query[0]][query[1]] += 1
            if query[2] < n-1:
                ret[query[2]+1][query[1]] -= 1
            if query[3] < n-1:
                ret[query[0]][query[3]+1] -= 1
            if query[2] < n-1 and query[3] < n-1:
                ret[query[2]+1][query[3]+1] += 1
        
        for i in range(1, n):
            ret[0][i] += ret[0][i-1]
            ret[i][0] += ret[i-1][0]
        
        for i in range(1, n):
            for j in range(1, n):
                ret[i][j] += ret[i-1][j] + ret[i][j-1] - ret[i-1][j-1]
        return ret

# 6293. Count the Number of Good Subarrays

In [102]:
class Solution:
    def countGood(self, nums: List[int], k: int) -> int:
        cnt = Counter()
        ans = left = pairs = 0
        for x in nums:
            pairs += cnt[x]
            cnt[x] += 1
            while left < len(nums) and pairs - cnt[nums[left]] + 1 >= k:
                pairs -= cnt[nums[left]] - 1
                cnt[nums[left]] -= 1
                left += 1
            if pairs >= k:
                ans += left + 1
        return ans

In [30]:
class Solution:
    def countGood(self, nums: List[int], k: int) -> int:
        count = collections.defaultdict(int)
        sums = 0
        ret = 0
        left = 0
        for i, num in enumerate(nums):
            count[num] += 1
            sums += count[num] * (count[num]-1)//2 - (count[num]-1) * (count[num]-2)//2 
            while sums >= k and i>=left:
                ret += len(nums)-i
                num = nums[left]
                sums -= count[num] * (count[num]-1)//2 - (count[num]-1) * (count[num]-2)//2 
                count[nums[left]] -= 1
                left += 1
                
        while sums >= k:
            ret += 1
            num = nums[left]
            sums -= count[num] * (count[num]-1)//2 - (count[num]-1) * (count[num]-2)//2 
            count[nums[left]] -= 1
            left += 1        
        return ret

In [31]:
nums = [3,1,4,3,2,2,4]
k = 2

In [32]:
a = Solution().countGood(nums, k)
a

defaultdict(<class 'int'>, {3: 1}) 0 0 0
defaultdict(<class 'int'>, {3: 1, 1: 1}) 0 0 0
defaultdict(<class 'int'>, {3: 1, 1: 1, 4: 1}) 0 0 0
defaultdict(<class 'int'>, {3: 2, 1: 1, 4: 1}) 1 0 0
defaultdict(<class 'int'>, {3: 2, 1: 1, 4: 1, 2: 1}) 1 0 0
defaultdict(<class 'int'>, {3: 1, 1: 1, 4: 1, 2: 2}) 1 2 1
defaultdict(<class 'int'>, {3: 1, 1: 0, 4: 1, 2: 2}) 1 4 3


4

# 6294. Difference Between Maximum and Minimum Price Sum

In [98]:
n = 6
edges = [[0,1],[1,2],[1,3],[3,4],[3,5]]
price = [9,8,7,6,10,5]

In [101]:
a = Solution().maxOutput(n, edges, price)
a

[9, 8, 7, 6, 10, 5]


24

In [94]:
class Solution:
    def maxOutput(self, n: int, edges: List[List[int]], price: List[int]) -> int:
        inf = float("inf")
        graph = [[] for _ in range(n)]
        for edge in edges:
            graph[edge[0]].append(edge[1])
            graph[edge[1]].append(edge[0])
        
        for i in range(n):
            if len(graph[i]) == 1:
                start = i
                break
        
        ret = 0
        
        def dfs(i, pre):
            nonlocal ret
            maxs1, maxs2 = 0, 0
            for nxt in graph[i]:
                if nxt != pre:
                    s1, s2 = dfs(nxt, i)
                    if maxs2 == 0:
                        ret = max(ret, s1)
                    else:
                        ret = max(ret, maxs2+price[i]+s1)
                    ret = max(ret, s2+maxs1+price[i])
                    maxs1 = max(maxs1, s1)
                    maxs2 = max(maxs2, s2)
            if maxs1 == 0:
                return price[i], 0
            return maxs1+price[i], maxs2+price[i]
        
        dfs(start, -1)
        return ret

In [100]:
class Solution:
    def maxOutput(self, n: int, edges: List[List[int]], price: List[int]) -> int:
        # 建无向图
        g = [set() for _ in range(n)]
        for a, b in edges:
            g[a].add(b)
            g[b].add(a)
        # 额外要记录的节点信息
        s = price.copy()  # 当前最大带叶子的路径和
        t = [0] * n  # 当前最大不带叶子的路径和
        # 找出图中初始叶子节点，加入待处理的集合
        # 此处以队列为例，但换成栈、堆、有序或无序的集合都没有问题
        q = collections.deque(i for i, d in enumerate(g) if len(d) == 1)
        ans = 0
        while q:
            a = q.popleft()  # 叶子节点
            b = g[a].pop()  # 取上级节点
            g[b].remove(a)  # 并删除该边
            # 记录b节点子树中经过b的最长路径
            # 可能是当前最大不带叶子的路径和 + 当前带叶子的路径和
            # 或者是当前最大带叶子的路径和 + 当前不带叶子的路径和
            ans = max(ans, s[a] + t[b], t[a] + s[b])
            # 记录b节点的最大带叶子的路径和和不带叶子的路径和
            s[b] = max(s[b], s[a] + price[b])
            t[b] = max(t[b], t[a] + price[b])
            if len(g[b]) == 1:  # 上级节点变为叶子节点
                q.append(b)  # 则上级节点入队
            elif not g[b]:  # 因刚删的边，已仅剩最后一个孤立节点
                break  # 则结束
        print(price)
        return ans